<h1> 4. Metode odabira značajki </h1>

U ovoj datoteci probat ćemo dobiti bolji uvid u važnosti značajki pripremljenog skupa.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
## BRISE UPOZORENJA
import warnings
warnings.filterwarnings('ignore')

In [3]:
## UČITAVANJE TRAIN I TEST SKUPA

X_train = pd.read_hdf('X_train.h5' , key = 'X_train')
X_test = pd.read_hdf('X_test.h5', key = 'X_test')
y_train = X_train['PRIJEVREMENI_RASKID'].ravel()
X_train.drop(['PRIJEVREMENI_RASKID'], axis = 1, inplace = True)

## to je normalizirani skup, spreman za treniranje

<h3> Univarijatni odabir </h3>

Makroekonomski indikatori su znacajke koje pocinju sa 'ind'. Imena tih značajki su izmijenjena u bilježnici "data_preparation". 
Prava imena su:
1. ind1-poc = GDP - year-on-year rate of growth (in %, constant prices)
2. ind2-poc = GDP - year-on-year rate of growth (in %, constant prices) 
3. ind3-poc = Average CPI year-on-year inflation rate 
4. ind4-poc = Current account balance (as of % GDP) 
5. ind5-poc = Exports of goods and services (as of % GDP)
6. ind6-poc = Imports of goods and services (as of % GDP) 
7. ind7-poc = External debt (as of % GDP) 
8. ind8-poc = Average exchange rate (HRK : 1 EUR)
9. ind9-poc = Average exchange rate (HRK : 1 USD)
10. ind10-poc = Consolidated general government net lending (+)/borrowing (-) (as % of GDP)
11. ind11-poc = General government debt (as % of GDP)
12. ind12-poc = Employment rate (ILO, persons above 15 years of age)

In [5]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

X = X_train
y = y_train  #target column i.e price range


from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
model = ExtraTreesClassifier()
model.fit(X,y)
print(model.feature_importances_) #use inbuilt class feature_importances of tree based classifiers
#plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(20).plot(kind='barh')
plt.show()

[0.08555687 0.08674948 0.08530122 0.08497143 0.13081771 0.36091176
 0.00515597 0.00357148 0.00211353 0.01070719 0.00262025 0.00182788
 0.00297936 0.00307195 0.00176933 0.00351347 0.00572132 0.00228124
 0.015418   0.02065043 0.01862848 0.03894018 0.0038191  0.02290237]


In [6]:
import xgboost as xgb

X = X_train
y = y_train  #target column i.e price range

import matplotlib.pyplot as plt
model = xgb.XGBClassifier()
model.fit(X,y)
print(model.feature_importances_) #use inbuilt class feature_importances of tree based classifiers
#plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(20).plot(kind='barh')
plt.show()

[5.2216128e-03 1.3731325e-02 0.0000000e+00 0.0000000e+00 7.8274995e-02
 1.0103937e-01 4.5812480e-02 1.4912061e-02 0.0000000e+00 2.0847201e-01
 3.1245418e-02 1.6995095e-02 3.2663171e-04 2.1549273e-02 4.0924281e-02
 0.0000000e+00 2.9804524e-02 1.2748811e-04 1.7806290e-02 0.0000000e+00
 1.7264122e-01 2.0405831e-02 9.8674096e-02 8.2036003e-02]


In [18]:
ft_num = X_train.shape[1]
print('Broj značajki u originalnom skupu:', ft_num)

Broj značajki u originalnom skupu: 24


In [8]:
X.columns

Index(['KLIJENT_ID', 'UGOVORENI_IZNOS', 'VISINA_KAMATE', 'STAROST',
       'PRODULJENJA', 'PLANIRANA_DULJINA_KREDITA', 'ind1-poc', 'ind2-poc',
       'ind3-poc', 'ind4-poc', 'ind5-poc', 'ind6-poc', 'ind7-poc', 'ind8-poc',
       'ind9-poc', 'ind10-poc', 'ind11-poc', 'ind12-poc',
       'VRSTA_KLIJENTA_CODE', 'VALUTA_CODE', 'TIP_KAMATE_CODE',
       'PROIZVOD_CODE', 'GODINA_OTVARANJA_CODE', 'VRSTA_PROIZVODA_CODE'],
      dtype='object')

Odabrane su tri različite varijante odabira značajki na kojima ćemo trenirati model. 

In [9]:
features1 = ['PLANIRANA_DULJINA_KREDITA', 'PRODULJENJA', 'VISINA_KAMATE', 'STAROST', 'KLIJENT_ID', 'UGOVORENI_IZNOS',
            'VALUTA_CODE', 'PROIZVOD_CODE', 'VRSTA_PROIZVODA_CODE', 'VRSTA_KLIJENTA_CODE', 'TIP_KAMATE_CODE', 'ind8-poc', 
             'ind3-poc', 'ind4-poc']

features2 = ['PLANIRANA_DULJINA_KREDITA', 'PRODULJENJA', 'KLIJENT_ID', 'UGOVORENI_IZNOS', 'STAROST', 'VISINA_KAMATE',
            'VALUTA_CODE', 'PROIZVOD_CODE', 'VRSTA_PROIZVODA_CODE', 'VRSTA_KLIJENTA_CODE', 'TIP_KAMATE_CODE', 'ind4-poc', 
             'ind11-poc']

features3 = ['ind4-poc', 'TIP_KAMATE_CODE', 'PLANIRANA_DULJINA_KREDITA', 'GODINA_OTVARANJA_CODE', 'VRSTA_PROIZVODA_CODE', 
             'PRODULJENJA', 'ind1-poc', 'ind9-poc', 'ind5-poc', 'ind11-poc', 'ind8-poc', 'PROIZVOD_CODE', 
             'VRSTA_KLIJENTA_CODE', 'ind6-poc', 'ind2-poc', 'UGOVORENI_IZNOS', 'KLIJENT_ID']

<h3> Korelacija značajki </h3>

U ovom dijelu promatramo kako su značajke koreliranje s ciljnom značajkom.

In [10]:
X = X_train.copy()
X['PRIJEVREMENI_RASKID'] = y_train

In [11]:
import seaborn as sns
#get correlations of each features in dataset
data = X
corrmat = data.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(20,20))
#plot heat map
g=sns.heatmap(data[top_corr_features].corr(),annot=True,cmap="RdYlGn")

<h3> Ispis skupa za učenje i testiranje s odabranim značajkama </h3>

In [12]:
X_train_out1 = X_train[features1]
X_train_out1['PRIJEVREMENI_RASKID'] = y_train

X_train_out2 = X_train[features2]
X_train_out2['PRIJEVREMENI_RASKID'] = y_train

X_train_out3 = X_train[features3]
X_train_out3['PRIJEVREMENI_RASKID'] = y_train

In [13]:
X_test_out1 = X_test[features1]
X_test_out2 = X_test[features2]
X_test_out3 = X_test[features3]

In [14]:
X_train_out1.to_hdf('X_train_ft1.h5', key= 'X_train_ft1')
X_train_out2.to_hdf('X_train_ft2.h5', key= 'X_train_ft2')
X_train_out3.to_hdf('X_train_ft3.h5', key= 'X_train_ft3')

X_train.to_hdf('X_train_noft.h5', key = 'X_train_noft') # sa svim znacajkama

In [15]:
X_test_out1.to_hdf('X_test_ft1.h5', key= 'X_test_ft1')
X_test_out2.to_hdf('X_test_ft2.h5', key= 'X_test_ft2')
X_test_out3.to_hdf('X_test_ft3.h5', key= 'X_test_ft3')

X_test.to_hdf('X_test_noft.h5', key = 'X_test_noft') # sa svim znacajkama